# Explore here

In [21]:
# Your code here
import pandas as pd
import numpy as np
import regex as re
archivo = 'https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv'
total_data  = pd.read_csv(archivo)
total_data.head()


,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [22]:
print(total_data.columns)
total_data.head() 

Index(['url', 'is_spam'], dtype='object')


,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [23]:
print(total_data.head(10))

                                                 url  is_spam
0  https://briefingday.us8.list-manage.com/unsubs...     True
1                             https://www.hvper.com/     True
2                 https://briefingday.com/m/v4n3i4f3     True
3   https://briefingday.com/n/20200618/m#commentform    False
4                        https://briefingday.com/fan     True
5  https://www.brookings.edu/interactives/reopeni...    False
6  https://www.reuters.com/investigates/special-r...    False
7  https://www.theatlantic.com/magazine/archive/2...    False
8  https://www.vox.com/2020/6/17/21294680/john-bo...    False
9  https://www.theguardian.com/travel/2020/jun/18...    False


In [24]:
total_data["is_spam_int"] = total_data["is_spam"].astype(int)
print(total_data.head())

                                                 url  is_spam  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...     True            1
1                             https://www.hvper.com/     True            1
2                 https://briefingday.com/m/v4n3i4f3     True            1
3   https://briefingday.com/n/20200618/m#commentform    False            0
4                        https://briefingday.com/fan     True            1


transformar en 1 y 0 

In [25]:

total_data.drop(columns=['is_spam'], inplace=True)


print(total_data.head())

                                                 url  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...            1
1                             https://www.hvper.com/            1
2                 https://briefingday.com/m/v4n3i4f3            1
3   https://briefingday.com/n/20200618/m#commentform            0
4                        https://briefingday.com/fan            1


eliminar los duplicados 

In [26]:

print(f"Forma del DataFrame antes de eliminar duplicados: {total_data.shape}")

total_data.drop_duplicates(inplace=True)

total_data.reset_index(inplace=True, drop=True)
print(f"Forma del DataFrame después de eliminar duplicados: {total_data.shape}")

print(total_data.head())

Forma del DataFrame antes de eliminar duplicados: (2999, 2)
Forma del DataFrame después de eliminar duplicados: (2369, 2)
                                                 url  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...            1
1                             https://www.hvper.com/            1
2                 https://briefingday.com/m/v4n3i4f3            1
3   https://briefingday.com/n/20200618/m#commentform            0
4                        https://briefingday.com/fan            1


Procesamiento del texto

In [27]:
def preprocess_text(text):
    
    text = re.sub(r'[^a-z ]', " ", text)

   
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    
    text = re.sub(r'\s+', " ", text.lower())

   
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

In [28]:
total_data["url"] = total_data["url"].apply(preprocess_text)
total_data.head()

,url,is_spam_int
0,"[https, briefingday, us, list, manage, com, un...",1
1,"[https, www, hvper, com]",1
2,"[https, briefingday, com, v, i]",1
3,"[https, briefingday, com, m, commentform]",0
4,"[https, briefingday, com, fan]",1


Lematizacion 

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('testing')
except LookupError:
    nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized_tokens)

total_data["url_lemmatized"] = total_data["url"].apply(lemmatize_tokens)


print(total_data[['url', 'url_lemmatized']].head())

                                                 url  \
0  [https, briefingday, us, list, manage, com, un...   
1                           [https, www, hvper, com]   
2                    [https, briefingday, com, v, i]   
3          [https, briefingday, com, m, commentform]   
4                     [https, briefingday, com, fan]   

                                   url_lemmatized  
0  http briefingday u list manage com unsubscribe  
1                              http www hvper com  
2                        http briefingday com v i  
3              http briefingday com m commentform  
4                        http briefingday com fan  


tokenización

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer


tokens_list = total_data["url"]
tokens_list = [" ".join(tokens) for tokens in tokens_list]

vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)

X = vectorizer.fit_transform(tokens_list).toarray()

y = total_data["is_spam_int"]


print("Primeras 5 filas de la matriz de características X:")
print(X[:5])


print("\nForma de la matriz de características X:", X.shape)
print("Forma del vector objetivo y:", y.shape)

Primeras 5 filas de la matriz de características X:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Forma de la matriz de características X: (2369, 691)
Forma del vector objetivo y: (2369,)


Modelo 

In [31]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (1895, 691)
Forma de X_test: (474, 691)
Forma de y_train: (1895,)
Forma de y_test: (474,)


In [32]:
from sklearn.svm import SVC

model = SVC(kernel="linear", random_state=42)

model.fit(X_train, y_train)

print("Modelo SVM con kernel lineal entrenado.")

Modelo SVM con kernel lineal entrenado.


In [33]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)


print("Predicciones del modelo SVM en el conjunto de prueba:")
print(y_pred)

Predicciones del modelo SVM en el conjunto de prueba:
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [34]:
from sklearn.metrics import accuracy_score

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo SVM (kernel lineal) en el conjunto de prueba: {accuracy:.4f}")

Precisión del modelo SVM (kernel lineal) en el conjunto de prueba: 0.9599


In [35]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('testing')
except LookupError:
    nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'[^a-z ]', " ", text)
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\s+', " ", text.lower())
    text = re.sub("&lt;/?.*?&gt;"," <> ", text)
    return text.split()

def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens  # Devolvemos una lista de tokens

# Nuevo texto a predecir
new_message = "To receive the €350 payment, the referred person must apply with your referral code. They must be completely new to our database and have no previous sales or contact experience."

# Aplicar preprocesamiento
processed_message = preprocess_text(new_message)

# Aplicar lematización a la lista de tokens
lemmatized_message = lemmatize_tokens(processed_message)

# Unir tokens en una sola cadena para vectorizar
lemmatized_message_str = " ".join(lemmatized_message)

# Vectorizar utilizando el vectorizador entrenado (asegúrate de que 'vectorizer' esté definido y entrenado)
try:
    vectorized_message = vectorizer.transform([lemmatized_message_str]).toarray()

    # Realizar predicción
    prediction = model.predict(vectorized_message)

    # Mostrar resultado
    print("Predicción:", "Spam" if prediction[0] == 1 else "No Spam")

except NameError as e:
    print(f"Error: {e}. Asegúrate de que 'vectorizer' y 'model' estén definidos y entrenados en tu sesión.")

Predicción: No Spam


In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report  


param_grid = {'C': [0.01, 0.1, 1, 10, 100]}


svm_model = SVC(kernel='linear', random_state=42)


grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_


y_pred_best = best_svm_model.predict(X_test)


best_accuracy = accuracy_score(y_test, y_pred_best)
best_report = classification_report(y_test, y_pred_best)

print("\nMejores hiperparámetros encontrados:", grid_search.best_params_)
print(f"\nAccuracy del mejor modelo SVM (kernel lineal) en el conjunto de prueba: {best_accuracy:.4f}")
print("\nClassification Report del mejor modelo SVM (kernel lineal):")
print(best_report)

Fitting 5 folds for each of 5 candidates, totalling 25 fits



Mejores hiperparámetros encontrados: {'C': 1}

Accuracy del mejor modelo SVM (kernel lineal) en el conjunto de prueba: 0.9599

Classification Report del mejor modelo SVM (kernel lineal):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       435
           1       0.95      0.54      0.69        39

    accuracy                           0.96       474
   macro avg       0.96      0.77      0.83       474
weighted avg       0.96      0.96      0.95       474



In [ ]:
import joblib
import os

# Definir la ruta de la carpeta donde guardar el modelo
model_folder = 'modelos'
os.makedirs(model_folder, exist_ok=True)
model_path = os.path.join(model_folder, 'spam_url_svm_optimized.joblib')
vectorizer_path = os.path.join(model_folder, 'tfidf_vectorizer.joblib')

# Guardar el mejor modelo SVM optimizado
joblib.dump(best_svm_model, model_path)
print(f"\nModelo SVM optimizado guardado en: {model_path}")


joblib.dump(vectorizer, vectorizer_path)
print(f"Vectorizador TF-IDF guardado en: {vectorizer_path}")


Modelo SVM optimizado guardado en: modelos/spam_url_svm_optimized.joblib
Vectorizador TF-IDF guardado en: modelos/tfidf_vectorizer.joblib
